# 🎓 Teaching Assistant AI Agent

## Overview
This is an intelligent Teaching Assistant Agent that can:
- Generate quizzes and assignments on any topic
- Check and grade student submissions
- Provide detailed feedback
- Track student progress
- Generate study materials
- Analyze code submissions

## Features
- **Memory Management**: Maintains conversation context
- **Multiple Tools**: 6+ custom tools for teaching tasks
- **Error Handling**: Robust error management
- **Structured Output**: Clean JSON-based tool invocations

## Setup Instructions
1. Get an API key from [OpenRouter](https://openrouter.ai/)
2. Run the setup cell and enter your API key
3. Execute all cells in order
4. Start interacting with the agent!

---

## 📦 Part 1: Setup and Environment Configuration

In [23]:
# Install required libraries
!pip install litellm -q

In [24]:
# Import necessary libraries
import os
import json
import re
from typing import List, Dict, Any
from datetime import datetime
from litellm import completion
import random

# Set up API key for OpenRouter
# Get your key from: https://openrouter.ai/
# ** IMPORTANT: Replace this with your actual OpenRouter API key **
api_key = input("Enter your OpenRouter API key (get it from https://openrouter.ai/): ").strip()
if not api_key:
    print("⚠️  Using placeholder key - you may need to provide a valid API key")
    api_key = "sk-or-v1-placeholder"  # Placeholder

os.environ["OPENROUTER_API_KEY"] = api_key

print("✅ Environment configured successfully!")
print(f"API Key set (first 20 chars): {api_key[:20]}...")


✅ Environment configured successfully!
API Key set (first 20 chars): sk-or-v1-dcc5c1f56d6...


## 🔧 Part 2: Core Functions

In [25]:
def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response using OpenRouter API"""
    try:
        api_key = os.environ.get("OPENROUTER_API_KEY")
        if not api_key or api_key == "sk-or-v1-placeholder":
            return "❌ Error: Valid OpenRouter API key not set. Please get a key from https://openrouter.ai/ and run the setup cell again."
        
        response = completion(
            model="openrouter/openai/gpt-4o",
            api_key=api_key,
            messages=messages,
            max_tokens=2048,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        error_msg = str(e)
        if "authentication" in error_msg.lower() or "api_key" in error_msg.lower():
            return f"❌ Authentication Error: Your OpenRouter API key may be invalid. Error: {error_msg}\n\nGet a valid key from: https://openrouter.ai/"
        else:
            return f"Error generating response: {error_msg}"

def extract_markdown_block(text: str, block_type: str) -> str:
    """Extract content from markdown code blocks"""
    pattern = f"```{block_type}\\s*\\n(.*?)\\n```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

def parse_action(response: str) -> Dict:
    """Parse the LLM response into a structured action dictionary"""
    try:
        # Extract action block
        action_text = extract_markdown_block(response, "action")
        response_json = json.loads(action_text)
        
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {
                "tool_name": "error",
                "args": {"message": "Response must contain 'tool_name' and 'args' fields."}
            }
    except json.JSONDecodeError as e:
        return {
            "tool_name": "error",
            "args": {"message": f"Invalid JSON response: {str(e)}. Please respond with valid JSON."}
        }
    except Exception as e:
        return {
            "tool_name": "error",
            "args": {"message": f"Error parsing response: {str(e)}"}
        }

print("✅ Core functions defined!")


✅ Core functions defined!


## 🛠️ Part 3: Custom Tools for Teaching Assistant

In [26]:
# Global storage for the teaching assistant
class TeachingAssistantStorage:
    def __init__(self):
        self.quizzes = {}
        self.assignments = {}
        self.student_submissions = {}
        self.grades = {}
        self.study_materials = {}
    
    def reset(self):
        """Reset all storage"""
        self.__init__()

# Initialize storage
storage = TeachingAssistantStorage()

# Tool 1: Generate Quiz
def generate_quiz(topic: str, num_questions: int = 5, difficulty: str = "medium") -> Dict:
    """
    Generate a quiz on a specific topic
    
    Args:
        topic: The subject/topic for the quiz
        num_questions: Number of questions to generate
        difficulty: Difficulty level (easy, medium, hard)
    
    Returns:
        Dictionary containing quiz data
    """
    try:
        quiz_id = f"quiz_{len(storage.quizzes) + 1}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
        
        # Generate quiz questions using LLM
        quiz_prompt = [
            {"role": "system", "content": "You are an expert educator creating assessment questions."},
            {"role": "user", "content": f"""Create {num_questions} {difficulty} level multiple-choice questions about {topic}.
            
Format as JSON:
{{
  "questions": [
    {{
      "question": "Question text",
      "options": ["A) option1", "B) option2", "C) option3", "D) option4"],
      "correct_answer": "A",
      "explanation": "Why this is correct"
    }}
  ]
}}"""}
        ]
        
        response = generate_response(quiz_prompt)
        
        # Extract JSON from response
        json_match = re.search(r'```json\s*\n(.*?)\n```', response, re.DOTALL)
        if json_match:
            quiz_data = json.loads(json_match.group(1))
        else:
            quiz_data = json.loads(response)
        
        # Store quiz
        storage.quizzes[quiz_id] = {
            "topic": topic,
            "difficulty": difficulty,
            "questions": quiz_data["questions"],
            "created_at": datetime.now().isoformat()
        }
        
        return {
            "success": True,
            "quiz_id": quiz_id,
            "topic": topic,
            "num_questions": len(quiz_data["questions"]),
            "difficulty": difficulty,
            "preview": quiz_data["questions"][:2]  # Show first 2 questions as preview
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# Tool 2: Generate Assignment
def generate_assignment(topic: str, assignment_type: str = "essay", requirements: str = "") -> Dict:
    """
    Generate an assignment on a specific topic
    
    Args:
        topic: The subject/topic for the assignment
        assignment_type: Type of assignment (essay, coding, problem_set, project)
        requirements: Specific requirements or constraints
    
    Returns:
        Dictionary containing assignment data
    """
    try:
        assignment_id = f"assignment_{len(storage.assignments) + 1}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
        
        # Generate assignment using LLM
        assignment_prompt = [
            {"role": "system", "content": "You are an expert educator creating meaningful assignments."},
            {"role": "user", "content": f"""Create a {assignment_type} assignment about {topic}.
            
Requirements: {requirements if requirements else 'Standard academic requirements'}

Include:
1. Clear instructions
2. Learning objectives
3. Deliverables
4. Grading rubric
5. Estimated time

Format as structured text."""}
        ]
        
        response = generate_response(assignment_prompt)
        
        # Store assignment
        storage.assignments[assignment_id] = {
            "topic": topic,
            "type": assignment_type,
            "requirements": requirements,
            "content": response,
            "created_at": datetime.now().isoformat()
        }
        
        return {
            "success": True,
            "assignment_id": assignment_id,
            "topic": topic,
            "type": assignment_type,
            "content": response
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# Tool 3: Grade Quiz
def grade_quiz(quiz_id: str, student_answers: Dict[int, str], student_name: str = "Student") -> Dict:
    """
    Grade a student's quiz submission
    
    Args:
        quiz_id: ID of the quiz
        student_answers: Dictionary mapping question index to answer (e.g., {0: 'A', 1: 'B'})
        student_name: Name of the student
    
    Returns:
        Dictionary containing grade and feedback
    """
    try:
        if quiz_id not in storage.quizzes:
            return {"success": False, "error": f"Quiz {quiz_id} not found"}
        
        quiz = storage.quizzes[quiz_id]
        questions = quiz["questions"]
        
        correct_count = 0
        results = []
        
        for idx, question in enumerate(questions):
            student_answer = student_answers.get(idx, "No answer")
            correct_answer = question["correct_answer"]
            is_correct = student_answer == correct_answer
            
            if is_correct:
                correct_count += 1
            
            results.append({
                "question_num": idx + 1,
                "student_answer": student_answer,
                "correct_answer": correct_answer,
                "is_correct": is_correct,
                "explanation": question.get("explanation", "")
            })
        
        score = (correct_count / len(questions)) * 100
        grade = "A" if score >= 90 else "B" if score >= 80 else "C" if score >= 70 else "D" if score >= 60 else "F"
        
        # Store grade
        grade_id = f"grade_{quiz_id}_{student_name}"
        storage.grades[grade_id] = {
            "quiz_id": quiz_id,
            "student_name": student_name,
            "score": score,
            "grade": grade,
            "results": results,
            "graded_at": datetime.now().isoformat()
        }
        
        return {
            "success": True,
            "student_name": student_name,
            "quiz_id": quiz_id,
            "score": round(score, 2),
            "grade": grade,
            "correct_answers": correct_count,
            "total_questions": len(questions),
            "detailed_results": results
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# Tool 4: Grade Assignment
def grade_assignment(assignment_id: str, submission_text: str, student_name: str = "Student") -> Dict:
    """
    Grade a student's assignment submission using AI
    
    Args:
        assignment_id: ID of the assignment
        submission_text: Student's submitted work
        student_name: Name of the student
    
    Returns:
        Dictionary containing grade and detailed feedback
    """
    try:
        if assignment_id not in storage.assignments:
            return {"success": False, "error": f"Assignment {assignment_id} not found"}
        
        assignment = storage.assignments[assignment_id]
        
        # Use LLM to grade the assignment
        grading_prompt = [
            {"role": "system", "content": "You are an experienced educator grading student work fairly and constructively."},
            {"role": "user", "content": f"""Grade this assignment submission.

ASSIGNMENT:
{assignment['content']}

STUDENT SUBMISSION:
{submission_text}

Provide:
1. Score (0-100)
2. Letter grade (A-F)
3. Strengths (3-5 points)
4. Areas for improvement (3-5 points)
5. Overall feedback

Format as JSON:
{{
  "score": 85,
  "grade": "B",
  "strengths": ["point1", "point2"],
  "improvements": ["point1", "point2"],
  "feedback": "Overall assessment"
}}"""}
        ]
        
        response = generate_response(grading_prompt)
        
        # Extract JSON from response
        json_match = re.search(r'```json\s*\n(.*?)\n```', response, re.DOTALL)
        if json_match:
            grading_data = json.loads(json_match.group(1))
        else:
            grading_data = json.loads(response)
        
        # Store grade
        grade_id = f"grade_{assignment_id}_{student_name}"
        storage.grades[grade_id] = {
            "assignment_id": assignment_id,
            "student_name": student_name,
            "submission": submission_text,
            "grading_data": grading_data,
            "graded_at": datetime.now().isoformat()
        }
        
        return {
            "success": True,
            "student_name": student_name,
            "assignment_id": assignment_id,
            **grading_data
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# Tool 5: Analyze Code
def analyze_code(code: str, language: str = "python") -> Dict:
    """
    Analyze code for quality, bugs, and improvements
    
    Args:
        code: The code to analyze
        language: Programming language
    
    Returns:
        Dictionary containing analysis results
    """
    try:
        analysis_prompt = [
            {"role": "system", "content": "You are an expert code reviewer and software engineer."},
            {"role": "user", "content": f"""Analyze this {language} code:

```{language}
{code}
```

Provide:
1. Code quality score (0-100)
2. Bugs or errors found
3. Best practices violations
4. Suggested improvements
5. Security concerns (if any)

Format as JSON."""}
        ]
        
        response = generate_response(analysis_prompt)
        
        return {
            "success": True,
            "language": language,
            "analysis": response,
            "analyzed_at": datetime.now().isoformat()
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# Tool 6: Generate Study Material
def generate_study_material(topic: str, material_type: str = "summary") -> Dict:
    """
    Generate study materials for students
    
    Args:
        topic: The topic to create materials for
        material_type: Type of material (summary, flashcards, notes, outline)
    
    Returns:
        Dictionary containing study materials
    """
    try:
        material_id = f"material_{len(storage.study_materials) + 1}"
        
        material_prompt = [
            {"role": "system", "content": "You are an expert educator creating effective study materials."},
            {"role": "user", "content": f"Create a {material_type} for the topic: {topic}"}
        ]
        
        response = generate_response(material_prompt)
        
        storage.study_materials[material_id] = {
            "topic": topic,
            "type": material_type,
            "content": response,
            "created_at": datetime.now().isoformat()
        }
        
        return {
            "success": True,
            "material_id": material_id,
            "topic": topic,
            "type": material_type,
            "content": response
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# Tool 7: Get Student Progress
def get_student_progress(student_name: str) -> Dict:
    """
    Get progress report for a specific student
    
    Args:
        student_name: Name of the student
    
    Returns:
        Dictionary containing student's progress
    """
    try:
        student_grades = [
            grade for grade in storage.grades.values()
            if grade.get("student_name") == student_name
        ]
        
        if not student_grades:
            return {
                "success": True,
                "student_name": student_name,
                "message": "No grades found for this student yet."
            }
        
        total_score = sum(g.get("score", 0) for g in student_grades)
        avg_score = total_score / len(student_grades)
        
        return {
            "success": True,
            "student_name": student_name,
            "total_assessments": len(student_grades),
            "average_score": round(avg_score, 2),
            "grades": student_grades
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# Tool 8: List Available Resources
def list_resources() -> Dict:
    """
    List all available quizzes, assignments, and materials
    
    Returns:
        Dictionary containing all available resources
    """
    return {
        "success": True,
        "quizzes": list(storage.quizzes.keys()),
        "assignments": list(storage.assignments.keys()),
        "study_materials": list(storage.study_materials.keys()),
        "total_grades": len(storage.grades)
    }

# Tool 9: Terminate
def terminate(message: str) -> Dict:
    """
    End the agent session
    
    Args:
        message: Final message to display
    
    Returns:
        Dictionary with termination info
    """
    return {
        "success": True,
        "message": message,
        "terminated": True
    }

print("✅ All 9 custom tools defined!")

✅ All 9 custom tools defined!


## 🤖 Part 4: Agent Behavior Definition

In [27]:
# Define agent rules and behavior
agent_rules = [{
    "role": "system",
    "content": """
You are an AI Teaching Assistant that helps educators and students with academic tasks.

AVAILABLE TOOLS:

1. generate_quiz(topic: str, num_questions: int, difficulty: str)
   - Create quizzes on any topic
   - difficulty: "easy", "medium", or "hard"
   - Returns quiz_id for later reference

2. generate_assignment(topic: str, assignment_type: str, requirements: str)
   - Create assignments
   - assignment_type: "essay", "coding", "problem_set", or "project"
   - Returns assignment_id

3. grade_quiz(quiz_id: str, student_answers: dict, student_name: str)
   - Grade quiz submissions
   - student_answers format: {0: "A", 1: "B", 2: "C"} (question index: answer)
   - Returns detailed grading results

4. grade_assignment(assignment_id: str, submission_text: str, student_name: str)
   - Grade assignment submissions with detailed feedback
   - Uses AI to evaluate quality, content, and structure

5. analyze_code(code: str, language: str)
   - Analyze code quality, find bugs, suggest improvements
   - language: "python", "javascript", "java", etc.

6. generate_study_material(topic: str, material_type: str)
   - Create study materials
   - material_type: "summary", "flashcards", "notes", or "outline"

7. get_student_progress(student_name: str)
   - Get comprehensive progress report for a student
   - Shows all grades and average score

8. list_resources()
   - List all available quizzes, assignments, and materials

9. terminate(message: str)
   - End the session with a summary message

BEHAVIOR RULES:
- Always be helpful, encouraging, and educational
- Provide clear explanations for grades and feedback
- When creating quizzes/assignments, ensure high quality and relevance
- Remember context from previous interactions
- If a user asks about their previous work, reference stored data
- Every response MUST include an action in the specified format

RESPONSE FORMAT:
First, think about what the user wants and which tool(s) to use.
Then respond with:

```action
{
  "tool_name": "tool_name_here",
  "args": {
    "arg1": "value1",
    "arg2": "value2"
  }
}
```

If the user's request is unclear, ask for clarification using the terminate tool.
"""
}]

print("✅ Agent behavior defined!")

✅ Agent behavior defined!


## 🔄 Part 5: Agent Loop Implementation

In [28]:
def execute_tool(action: Dict) -> Dict:
    """
    Execute a tool based on the parsed action
    
    Args:
        action: Parsed action dictionary with tool_name and args
    
    Returns:
        Result of the tool execution
    """
    tool_name = action.get("tool_name")
    args = action.get("args", {})
    
    try:
        if tool_name == "generate_quiz":
            return generate_quiz(**args)
        
        elif tool_name == "generate_assignment":
            return generate_assignment(**args)
        
        elif tool_name == "grade_quiz":
            return grade_quiz(**args)
        
        elif tool_name == "grade_assignment":
            return grade_assignment(**args)
        
        elif tool_name == "analyze_code":
            return analyze_code(**args)
        
        elif tool_name == "generate_study_material":
            return generate_study_material(**args)
        
        elif tool_name == "get_student_progress":
            return get_student_progress(**args)
        
        elif tool_name == "list_resources":
            return list_resources()
        
        elif tool_name == "terminate":
            return terminate(**args)
        
        elif tool_name == "error":
            return {"error": args.get("message", "Unknown error")}
        
        else:
            return {"error": f"Unknown tool: {tool_name}"}
    
    except Exception as e:
        return {"error": f"Error executing {tool_name}: {str(e)}"}

def run_teaching_assistant(max_iterations: int = 15):
    """
    Main agent loop for the Teaching Assistant
    
    Args:
        max_iterations: Maximum number of interaction loops
    """
    print("\n" + "="*70)
    print("🎓 TEACHING ASSISTANT AI AGENT")
    print("="*70)
    print("\nI can help you with:")
    print("  • Creating quizzes and assignments")
    print("  • Grading student work")
    print("  • Analyzing code")
    print("  • Generating study materials")
    print("  • Tracking student progress")
    print("\nType 'quit' or 'exit' to end the session.")
    print("="*70 + "\n")
    
    # Initialize memory with agent rules
    memory = agent_rules.copy()
    iterations = 0
    
    while iterations < max_iterations:
        try:
            # Get user input
            user_input = input("\n👤 You: ").strip()
            
            if not user_input:
                continue
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
                print("\n🎓 Agent: Goodbye! Thanks for using the Teaching Assistant. Have a great day!")
                break
            
            # Add user message to memory
            memory.append({"role": "user", "content": user_input})
            
            # Generate response from LLM
            print("\n🤔 Agent thinking...")
            response = generate_response(memory)
            print(f"\n🎓 Agent: {response}")
            
            # Parse action from response
            action = parse_action(response)
            print(f"\n🔧 Executing tool: {action['tool_name']}")
            
            # Execute the tool
            result = execute_tool(action)
            
            # Display result
            print(f"\n📊 Result:")
            print(json.dumps(result, indent=2))
            
            # Add assistant response and result to memory
            memory.append({"role": "assistant", "content": response})
            memory.append({"role": "user", "content": json.dumps(result)})
            
            # Check for termination
            if action["tool_name"] == "terminate" or result.get("terminated"):
                print("\n" + "="*70)
                print("Session ended. Thank you for using the Teaching Assistant!")
                print("="*70)
                break
            
            iterations += 1
            
        except KeyboardInterrupt:
            print("\n\n🎓 Agent: Session interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {str(e)}")
            print("Let's try again...\n")
    
    if iterations >= max_iterations:
        print("\n" + "="*70)
        print("Maximum iterations reached. Session ended.")
        print("="*70)

print("✅ Agent loop implementation ready!")

✅ Agent loop implementation ready!


## 🚀 Part 6: Run the Agent

In [29]:
# Run the Teaching Assistant Agent
# Make sure you've set your OpenRouter API key above!
run_teaching_assistant(max_iterations=5)


🎓 TEACHING ASSISTANT AI AGENT

I can help you with:
  • Creating quizzes and assignments
  • Grading student work
  • Analyzing code
  • Generating study materials
  • Tracking student progress

Type 'quit' or 'exit' to end the session.


🤔 Agent thinking...

🎓 Agent: To create a quiz on Python programming, I'll generate a medium difficulty quiz with 5 questions. Let's proceed with that.

```action
{
  "tool_name": "generate_quiz",
  "args": {
    "topic": "Python programming",
    "num_questions": 5,
    "difficulty": "medium"
  }
}
```

🔧 Executing tool: generate_quiz

📊 Result:
{
  "success": true,
  "quiz_id": "quiz_1_20260205174754",
  "topic": "Python programming",
  "num_questions": 5,
  "difficulty": "medium",
  "preview": [
    {
      "question": "What will be the output of the following Python code?\n\n```python\nx = [1, 2, 3, 4]\ny = x\nx.append(5)\nprint(y)\n```",
      "options": [
        "A) [1, 2, 3, 4]",
        "B) [1, 2, 3, 4, 5]",
        "C) [5]",
        "D) An 

## 📝 Example Interactions

Here are some example interactions you can try:

### Example 1: Create a Quiz
```
You: Create a medium difficulty quiz on Python programming with 5 questions
Agent: [Generates quiz and stores it]
```

### Example 2: Grade a Quiz
```
You: Grade the quiz quiz_1_xxx for student John with answers: {0: "A", 1: "B", 2: "C", 3: "A", 4: "D"}
Agent: [Grades the quiz and provides detailed feedback]
```

### Example 3: Analyze Code
```
You: Analyze this Python code: def add(a, b): return a + b
Agent: [Provides detailed code analysis]
```

### Example 4: Create Assignment
```
You: Create a coding assignment on object-oriented programming
Agent: [Generates comprehensive assignment]
```

### Example 5: Check Student Progress
```
You: Show me the progress for student John
Agent: [Displays all grades and average]
```

### Example 6: Generate Study Material
```
You: Create flashcards for learning SQL
Agent: [Generates study flashcards]
```

---

## 🧪 Testing Section

Run these cells to test individual tools:

In [30]:
# Test 1: Generate a quiz
print("Testing quiz generation...\n")
result = generate_quiz("Python basics", num_questions=3, difficulty="easy")
print(json.dumps(result, indent=2))

Testing quiz generation...

{
  "success": true,
  "quiz_id": "quiz_2_20260205175122",
  "topic": "Python basics",
  "num_questions": 3,
  "difficulty": "easy",
  "preview": [
    {
      "question": "What is the correct file extension for Python files?",
      "options": [
        "A) .txt",
        "B) .py",
        "C) .java",
        "D) .html"
      ],
      "correct_answer": "B",
      "explanation": "Python files use the .py extension to identify them as Python scripts."
    },
    {
      "question": "Which of the following is a valid way to start a comment in Python?",
      "options": [
        "A) //",
        "B) <!--",
        "C) #",
        "D) /*"
      ],
      "correct_answer": "C",
      "explanation": "In Python, comments are initiated by the '#' symbol."
    }
  ]
}


In [31]:
# Test 2: List available resources
print("Testing resource listing...\n")
result = list_resources()
print(json.dumps(result, indent=2))

Testing resource listing...

{
  "success": true,
  "quizzes": [
    "quiz_1_20260205174754",
    "quiz_2_20260205175122"
  ],
  "assignments": [
    "assignment_1_20260205174853"
  ],
  "study_materials": [
    "material_1"
  ],
  "total_grades": 0
}


In [32]:
# Test 3: Analyze code
print("Testing code analysis...\n")
test_code = """
def calculate_sum(numbers):
    total = 0
    for num in numbers:
        total = total + num
    return total
"""
result = analyze_code(test_code, "python")
print(json.dumps(result, indent=2))

Testing code analysis...

{
  "success": true,
  "language": "python",
  "analysis": "```json\n{\n  \"code_quality_score\": 85,\n  \"bugs_or_errors_found\": [],\n  \"best_practices_violations\": [\n    \"Variable name 'total' could be more descriptive.\"\n  ],\n  \"suggested_improvements\": [\n    \"Use the built-in sum() function for simplicity and readability: return sum(numbers).\",\n    \"Consider adding type hints for the function signature, e.g., def calculate_sum(numbers: list) -> int.\"\n  ],\n  \"security_concerns\": []\n}\n```",
  "analyzed_at": "2026-02-05T17:51:27.536888"
}


## 📚 Additional Features

### Reset Storage
If you want to start fresh, run this cell:

In [33]:
# Reset all stored data
storage.reset()
print("✅ Storage reset! All quizzes, assignments, and grades cleared.")

✅ Storage reset! All quizzes, assignments, and grades cleared.


## 🎯 Key Features Demonstrated

### ✅ Assignment Requirements Met:

1. **Custom Tools (9 total)**:
   - ✅ `generate_quiz()` - Create quizzes
   - ✅ `generate_assignment()` - Create assignments
   - ✅ `grade_quiz()` - Grade quizzes
   - ✅ `grade_assignment()` - Grade assignments
   - ✅ `analyze_code()` - Code analysis
   - ✅ `generate_study_material()` - Study materials
   - ✅ `get_student_progress()` - Progress tracking
   - ✅ `list_resources()` - List resources
   - ✅ `terminate()` - End session

2. **Agent Behavior**:
   - ✅ Clear system instructions
   - ✅ Tool documentation
   - ✅ Response format definition

3. **Memory Management**:
   - ✅ Conversation history maintained
   - ✅ Context preservation across turns
   - ✅ Reference to previous interactions

4. **Agent Loop**:
   - ✅ Continuous interaction
   - ✅ User input handling
   - ✅ Tool execution
   - ✅ Memory updates

5. **Error Handling**:
   - ✅ Invalid input handling
   - ✅ Tool error management
   - ✅ JSON parsing errors
   - ✅ Graceful failure recovery

6. **API Integration**:
   - ✅ OpenRouter API support
   - ✅ LiteLLM integration
   - ✅ GPT-4 model usage

---

## 💡 Tips for Best Results

1. **Be Specific**: When asking for quizzes or assignments, specify the topic clearly
2. **Use IDs**: Reference quiz_id and assignment_id when grading
3. **Check Resources**: Use `list_resources()` to see what's available
4. **Student Names**: Use consistent student names to track progress
5. **Code Analysis**: Provide complete, well-formatted code for best analysis

---

**Created by**: AI Teaching Assistant Team  
**Version**: 1.0  
**Last Updated**: 2026  
**License**: Educational Use